# Exercise 7: Optimal offering strategy of price-taker wind producer

## Imports

In [1]:
import gurobipy as gp
from gurobipy import GRB

## Statement

In [2]:
marginal_cost = 15  # EUR/MWh
lambda_DA = 20  # EUR/MWh, Day-Ahead Market price

# Probabilities and prices for balancing market
lambda_B_prices = [15, 35]  # EUR/MWh for balancing market
P_lambda_B = [0.5, 0.5]  # Probability of each price in balancing market

# Probabilities and production levels for wind production
P_W1_levels = [125, 75]  # MWh of wind production
P_PW1 = [0.5, 0.5]  # Probability of each production level


## Reolution 

### b)

In [3]:
def optimize_one_price():
    """Optimize expected profit under the one-price balancing scheme using Gurobi."""
    model = gp.Model("One_Price_Scheme")

    # Decision variable: x (amount offered in day-ahead market)
    x = model.addVar(name="x", vtype=GRB.CONTINUOUS, lb=0, ub=150)

    # Objective: Expected profit calculation
    expected_profit = 0
    for i, lambda_B in enumerate(lambda_B_prices):
        for j, PW1 in enumerate(P_W1_levels):
            probability = P_lambda_B[i] * P_PW1[j]

            # Revenue from day-ahead market
            revenue_DA = lambda_DA * x

            # Imbalance
            imbalance = PW1 - x

            # Revenue in the balancing market
            revenue_B = lambda_B * imbalance

            # Profit for this scenario
            profit = revenue_DA + revenue_B - (marginal_cost * PW1)
            expected_profit += probability * profit

    # Set the objective function
    model.setObjective(expected_profit, GRB.MAXIMIZE)

    # Optimize
    model.optimize()

    # Output results
    if model.status == GRB.OPTIMAL:
        optimal_x = x.X
        max_profit = model.ObjVal
        print("One-Price Scheme Results:")
        print(f"Optimal Offer (x): {optimal_x} MWh")
        print(f"Maximum Expected Profit: {max_profit} EUR")
    else:
        print("No optimal solution found for the one-price scheme.")

optimize_one_price()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-07
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD Ryzen 7 7840U with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0xca78f4d2
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [5e+00, 5e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.000000000e+03
One-Price Scheme Results:
Optimal Offer (x): 0.0 MWh
Maximum Expected Profit: 1000.0 EU

### c)

In [4]:
def optimize_two_price():
    """Optimize expected profit under the two-price balancing scheme using Gurobi."""
    model = gp.Model("Two_Price_Scheme")

    # Decision variable: x (amount offered in day-ahead market)
    x = model.addVar(name="x", vtype=GRB.CONTINUOUS, lb=0, ub=150)

    # Objective: Expected profit calculation
    expected_profit = 0
    for i, lambda_B in enumerate(lambda_B_prices):
        for j, PW1 in enumerate(P_W1_levels):
            probability = P_lambda_B[i] * P_PW1[j]

            # Revenue from day-ahead market
            revenue_DA = lambda_DA * x

            # Imbalance (difference between production and offer)
            imbalance = PW1 - x

            # Define auxiliary variables for overproduction and underproduction revenue
            overproduction_revenue = model.addVar(lb=0, name=f"overproduction_revenue_{i}_{j}")
            underproduction_cost = model.addVar(lb=0, name=f"underproduction_cost_{i}_{j}")

            # Constraints to define overproduction and underproduction revenue based on imbalance
            model.addConstr(overproduction_revenue >= lambda_B * imbalance)
            model.addConstr(overproduction_revenue >= 0)  # Only contribute when imbalance is positive

            model.addConstr(underproduction_cost >= -lambda_B * imbalance)
            model.addConstr(underproduction_cost >= 0)  # Only contribute when imbalance is negative

            # Total revenue from balancing market for this scenario
            revenue_B = overproduction_revenue - underproduction_cost

            # Total profit for this scenario
            cost = marginal_cost * PW1
            profit = revenue_DA + revenue_B - cost
            expected_profit += probability * profit

    # Set the objective function
    model.setObjective(expected_profit, GRB.MAXIMIZE)

    # Optimize
    model.optimize()

    # Output results
    if model.status == GRB.OPTIMAL:
        optimal_x = x.X
        max_profit = model.ObjVal
        print("\nTwo-Price Scheme Results:")
        print(f"Optimal Offer (x): {optimal_x} MWh")
        print(f"Maximum Expected Profit: {max_profit} EUR")
    else:
        print("No optimal solution found for the two-price scheme.")

optimize_two_price()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD Ryzen 7 7840U with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 9 columns and 24 nonzeros
Model fingerprint: 0xc3376753
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [2e+02, 2e+02]
  RHS range        [1e+03, 4e+03]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
No optimal solution found for the two-price scheme.
